In [ ]:
from keras.applications import MobileNet
model = MobileNet(weights = 'imagenet',include_top = False,input_shape=(224,224,3))
for layers in model.layers:
    print(layers.__class__.__name__)
model.layers[1].trainable
for layers in model.layers:
    layers.trainable = False
    
for (i,layer) in enumerate(model.layers):
    print(str(i)+" "+layer.__class__.__name__,layer.trainable)

In [ ]:
def newmodel(model):
        top_model = model.output
        top_model = GlobalAveragePooling2D()(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(512,activation='relu')(top_model)
        top_model = Dense(2,activation='softmax')(top_model)
        return top_model

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.models import Model
modify_mod = newmodel(model)
fin_model = Model(inputs=model.input,outputs=modify_mod)
fin_model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_data = 'dataset/train'
validation_data = 'dataset/validation'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=45,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size=32
train_generator = train_datagen.flow_from_directory(train_data,
                                                    target_size=(224,224),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_data,
                                                             target_size=(224,224),
                                                             batch_size=batch_size,
                                                             class_mode='categorical')

In [ ]:
from keras.optimizers import RMSprop

In [ ]:
fin_model.compile(optimizer=RMSprop(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
fin_model.fit_generator(train_generator,
                    steps_per_epoch= 200,
                    epochs=2,
                    validation_data=validation_generator,
                    validation_steps= 50)

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "manisha ", 
                      "[1]": "nadeem",}

monkey_breeds_dict_n = {"manisha": "manisha ", 
                      "nadeem": "nadeem",}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,20):
    input_im = getRandomImage("dataset/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(fin_model.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()